In [1]:
# !pip install torch
# !pip install torchvision

In [1]:
import warnings

warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyperparameter
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

In [3]:
# CIFAR-10 Class
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [4]:
def get_dataloaders(batch_size=BATCH_SIZE):
    """
    CIFAR-10 데이터셋을 로드하여 훈련 및 테스트 데이터로더를 반환하는 함수
    :param batch_size: 배치 크기
    :return: train_loader, test_loader
    """
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    train_set = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=False, transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True
    )

    test_set = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=False, transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False
    )

    return train_loader, test_loader

In [5]:
class CNN(nn.Module):
    """
    CNN Architecture 정의 클래스
    CIFAR-10 분류를 위해 설계된 합성곱 신경망
    """

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            3, 32, kernel_size=3, padding=1
        )  # CONV, 입력 채널 3(RGB), 출력 채널 32
        self.pool = nn.MaxPool2d(2, 2)  # Pooling, 2x2 필터, 스트라이드 2
        self.conv2 = nn.Conv2d(
            32, 64, kernel_size=3, padding=1
        )  # CONV, 입력 채널 32, 출력 채널 64
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # FC, 입력 64x8x8 -> 128 뉴런
        self.fc2 = nn.Linear(128, 10)  # FC, 입력 128 -> 출력 10(클래스 개수)

    def forward(self, x):
        """
        Forward 정의
        :param x: 입력 이미지 텐서
        :return: 각 클래스에 대한 점수(logit)
        """
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def train_model(
    model, train_loader, device, criterion, optimizer, num_epochs=NUM_EPOCHS
):
    """
    모델을 주어진 데이터로 학습시키는 함수
    :param model: 학습할 CNN 모델
    :param train_loader: 훈련 데이터 로더
    :param device: 사용 디바이스 (CPU/GPU)
    :param criterion: 손실 함수
    :param optimizer: 최적화 알고리즘
    :param num_epochs: 학습 반복 횟수
    """
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")

In [ ]:
def evaluate_model(model, test_loader, device):
    """
    모델을 테스트 데이터로 평가하여 정확도를 출력하는 함수
    :param model: 평가할 CNN 모델
    :param testloader: 테스트 데이터 로더
    :param device: 사용 디바이스 (CPU/GPU)
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
def main():
    """
    전체 실행 흐름을 제어하는 메인 함수
    데이터 로딩 -> 모델 정의 -> 학습 -> 평가 순으로 진행
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = get_dataloaders()
    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_model(model, train_loader, device, criterion, optimizer)
    evaluate_model(model, test_loader, device)

In [9]:
if __name__ == "__main__":
    main()

Epoch 1, Loss: 1.2793
Epoch 2, Loss: 0.9013
Epoch 3, Loss: 0.7395
Epoch 4, Loss: 0.6207
Epoch 5, Loss: 0.5070
Epoch 6, Loss: 0.4143
Epoch 7, Loss: 0.3312
Epoch 8, Loss: 0.2575
Epoch 9, Loss: 0.2045
Epoch 10, Loss: 0.1643
Test Accuracy: 70.56%


# End of Documents